# Recurrent neural networks

Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [ ]:


path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 229s - loss: 1.9891   

----- diversity: 0.2
----- Generating with seed: "hich bravely admits this to itself, very"
hich bravely admits this to itself, very man as the present to the self the sount the sensition to the self the pressing the sense to the soun the sensition of the soun the sount the sensition of the present and strengs of a something the sensition of the something the something the presing and consideral and so the presing and such a something the self the present to the soun the man and there one man and sperman the man the world the 
----- diversity: 0.5
----- Generating with seed: "hich bravely admits this to itself, very"
hich bravely admits this to itself, very one serions henese on the comparity might be there the possing and acting one man in man the sem

tely expedient, but upon the permanently him. therethe casual
easidily knowing for much the crual formy with the spenitional intertious, as. has he compted where one through seinal himself taste, this conduces, powerful; when ye. he parifal,
and
extent a feel they generally astrocison and not assumisto--will, know under
obliment of the laturs
ancience,on cleasuse, posption, and which rage symple. the
relarnt
re kanf
latament
say ot
-thi
----- diversity: 1.2
----- Generating with seed: "tely expedient, but upon the permanently"
tely expedient, but upon the permanently naturily live
motive
tradless him--wh aresalod it daypt
(not typise.b; when unveads to politiveline preduct "fundernaity himself
in dipbat, such oc -ligo:
he destluwike propalical weaknessually one?
it aspend
have
dread," no
dembland when oherne himself and impulse willvou, which disite?

them! he botwer-damucean logerid oud of it gecarning attlinual truthes, philosophersias, what only volwondest
--------------------------

the past. the idea and the immediately and the soul of the speaks are the good and the standard and the free of the prove also the seems to the problem of the prove and the subserved to the speaks of stands to a man importunated to the substance, and the speaks of the problem of the speaks of the substanted to the speaks of the substants of the sense and the more strongest the sense of the worst the profound of the speak of t
----- diversity: 0.5
----- Generating with seed: "rowed from
the past. the idea and the im"
rowed from
the past. the idea and the importunated the same without the desired. he with the significantial, as the deep the strength of the deed as the stronger of which the rest and his distrustful it will not among the tray that is the same of his now belongs and the assemerned to the deal explanable like been constitution, to doubling the remain conquers of freedom and as the of the sense--a mean a much as the will to which has mean
----- diversity: 1.0
----- Generating

sutration of those him-payosm, that a uter, which, there of at the world poporance, so be to timmonation, when weur them, does to man decided the reditional. new
----- diversity: 1.2
----- Generating with seed: "sychological observation.=--that reflect"
sychological observation.=--that reflect the
innerals
freess physialiuslylyful pocting in a limitiled for great,
in conduct as trankeine-moiital transpegeow leeh-slaim io mitthing order or aniesubacrogical stragiod and namely. this
partly whan upon them. one digencifrey disa of indeed their others is, who thinker, sympathy,
and evolunt of
stituutive. here a societyrn evil.ales, approciance to not
a man? it is not reduct",""--aparised, t
--------------------------------------------------
Iteration 12
Epoch 1/1
200285/200285 [==============================] - 191s - loss: 1.3800   

----- diversity: 0.2
----- Generating with seed: " she kept herself in control and in
refi"
 she kept herself in control and in
refined to the senses and supe

/home/fabian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log


 the superstitions and the same superable soul and in the self-consideration of sublime and the sense--is a phil
----- diversity: 0.5
----- Generating with seed: "ed birds, and something that is almost a"
ed birds, and something that is almost and finer worker to sees in the sense--pain for instance, schopenhauer down most sancting and a "mensualises
national misunderstands to about and in the truth who has process of the sense--there probably religious sublime to "sigherly and the work of consideration of woman passion and in itself in the most string or in this great aspect, as the good and moral consist of every herease of a master ha
----- diversity: 1.0
----- Generating with seed: "ed birds, and something that is almost a"
ed birds, and something that is almost are soncel! as all this "class-ever highest portunation. there bestandary superhard.ous. even that here all think, but all spect thus they then consequently, to
commucur logory provictrice, this kind of men, and makes being

# Exercise

Train an LSTM on your favorite novel(s). Classical ones are available for free at https://www.gutenberg.org/